<a href="https://colab.research.google.com/github/profteachkids/chetools/blob/main/sparse_jac_combineRows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sortedcontainers

In [2]:
from numba import njit
import numpy as np
from itertools import permutations
import scipy as sp
from sortedcontainers import SortedList, SortedDict, SortedSet
from collections import deque
from copy import deepcopy
np.set_printoptions(precision=1, linewidth=300, threshold=1e6, suppress= True)

In [17]:
Nv = 10
Nv_sizes = np.random.randint(low=2, high=5, size=Nv)
cum_pos = np.cumsum(Nv_sizes)
cum_pos = np.r_[0, cum_pos]
N = cum_pos[-1]

diag_data = [np.random.uniform(size=(Nv_sizes[i], Nv_sizes[i])) for i in range(Nv)]
Noff_diag = Nv
off_diag_blocks =np.asarray(tuple(set(tuple(permutations(tuple(range(Nv)),2))) - set(tuple([i,i]) for i in range(Nv))))
np.random.shuffle(off_diag_blocks)
off_diag_blocks=off_diag_blocks[:Noff_diag]
off_data=[np.random.uniform(size=(Nv_sizes[i], Nv_sizes[j])) for i,j in off_diag_blocks]

data = []
col = []
row=[]

for i in range(Nv):
    coo = sp.sparse.coo_array(diag_data[i])
    data.extend(coo.data)
    col.extend(coo.row+cum_pos[i])
    row.extend(coo.col+cum_pos[i])

for i in range(Noff_diag):
    coo = sp.sparse.coo_array(off_data[i])
    data.extend(coo.data)
    brow, bcol = off_diag_blocks[i]
    col.extend(coo.row+cum_pos[brow])
    row.extend(coo.col+cum_pos[bcol])

a = sp.sparse.coo_array((data, (row, col)),shape=(N,N))
nz = (a!=0).astype(np.int8)
c = (nz @ nz.T).astype(bool)

In [18]:
cand = set(range(N))
ls = []
while cand:
    k = cand.pop()
    l1=[k]
    cand2 = deepcopy(cand)
    v=c.indices[c.indptr[k]:c.indptr[k+1]]
    while True:
        for item in v:
            cand2.discard(item)
        if not(cand2):
            break
        first=cand2.pop()
        l1.append(first)
        v = c.indices[c.indptr[first]:c.indptr[first+1]]
        cand.remove(first)

    ls.append(l1)

In [22]:
adense=a.todense()
for l in ls:
    print(adense[l,:])
    print()

[[0.6 0.4 0.  0.  0.  0.  0.  0.  0.7 0.8 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.6 1.  0.2 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.3 0.3 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.9 0.9 0.7 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.4 0.6 0.  0.  0.1 0.8 0.2 0.1 0.  0.  0.  0.  0.  0.  0.  0.  0.3 0.2 0.6 0.3]]

[[0.4 0.9 0.1 0.  0.  0.  0.  0.  0.1 0.2 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.6 0.8 0.6 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.9 0.4 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  

In [20]:
ls[0]

[0, 5, 12, 22, 26]

In [21]:
len(ls)

11